In [14]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
%autoreload 2

In [17]:
from util import (
    bfs,
    unexplored_directions,
    find_unexplored_room,
    find_room_direction,
    movement,wise_wizard,
    pickle_graph, 
    record_room_info, 
    pickup_treasure
)

from decouple import config
import requests
import time
import pickle


In [27]:
API_KEY=config("API_KEY")
init_headers={"Authorization":f"Token {API_KEY}"}
response = requests.get("https://lambda-treasure-hunt.herokuapp.com/api/adv/init/",headers=init_headers)
room_id = record_room_info(player_graph,response)

KeyError: None

In [8]:
room_id = response.json()["room_id"]
len(player_graph),room_id

(212, 2)

In [ ]:
while len(player_graph) < 500:
    
#     import pdb;
#     pdb.set_trace()
    
    print(room_id,len(player_graph))
    
    direction = None
    ud  = unexplored_directions(player_graph,room_id)
    
    
    cooldown = response.json()["cooldown"]
    time.sleep(cooldown)
    
    if len(response.json()["items"])>0:
        for ii in response.json()["items"]:
            response = pickup_treasure(ii,API_KEY)
            print(response.json()["messages"])
            time.sleep(response.json()["cooldown"])
    
    previous_room = room_id
    
    
    
    if len(response.json()["exits"])==1:
        direction = response.json()["exits"].pop()
        response = wise_wizard(direction,previous_room,API_KEY)
        
    elif len(ud)>=1:
        direction = ud.pop()
        response = movement(direction,API_KEY)
        
    elif len(ud) ==0:
        nr = find_unexplored_room(player_graph)
        path = bfs(room_id, nr, player_graph)
        path = path[1:]
        while len(path)>0:
            next_room = path.pop(0)
            direction = find_room_direction(response,player_graph, room_id, next_room)
            if len(path)>=0:
                previous_room = room_id
                response = wise_wizard(direction,next_room,API_KEY)
                record_room_info(player_graph,response,direction=direction,previous_room=previous_room)
                break
    pickle_graph(player_graph)

    
    room_id = record_room_info(player_graph,response,direction=direction,previous_room=previous_room)
    

47 215
71 216
47 216
43 216
10 216
19 217
77 218
19 218
20 219
46 220
62 221
84 222
91 223
99 224
146 225
257 226
[]
364 227
['You have picked up tiny treasure']
['You have picked up tiny treasure']
[]
448 228
['You have picked up tiny treasure']
['You have picked up tiny treasure']
364 228
381 229
['You have picked up tiny treasure']
394 230
['You have picked up tiny treasure']
['You have picked up small treasure']
['You have picked up tiny treasure']
['You have picked up tiny treasure']
381 230


In [29]:
response.json()

{'room_id': 47,
 'title': 'A misty room',
 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.',
 'coordinates': '(58,61)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': [],
 'items': [],
 'exits': ['n', 'e'],
 'cooldown': 1.0,
 'errors': [],
 'messages': []}